In [1]:
import os
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('..')

import numpy as np
import random
import torch

from pytorch_pretrained_bert.tokenization import BertTokenizer

from lib import data_processors, tasks
from lib.bert import BertForSequenceClassification
from lib.train_eval import train, evaluate, predict

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
%env CUDA_VISIBLE_DEVICES=0

params = {
    'data_dir': '../datasets/QQP',
    'output_dir': '../output',
    'cache_dir': '../model_cache',
    'task_name': 'qqp',
    'bert_model': 'bert-base-uncased',
    'max_seq_length': 128,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'learning_rate': 2e-5,
    'warmup_proportion': 0.1,
    'num_train_epochs': 1,
    'seed': 1331,
    'device': torch.device(
        'cuda' if torch.cuda.is_available()
        else 'cpu')
}

random.seed(params['seed'])
np.random.seed(params['seed'])
torch.manual_seed(params['seed'])

env: CUDA_VISIBLE_DEVICES=0


In [3]:
params['num_labels'] = tasks.num_labels[params['task_name']]
params['label_list'] = tasks.label_lists[params['task_name']]

processor = tasks.processors[params['task_name']]()
tokenizer = BertTokenizer.from_pretrained(
    params['bert_model'], do_lower_case=True)

train_examples = processor.get_train_examples(params['data_dir'])
dev_examples = processor.get_dev_examples(params['data_dir'])

model = BertForSequenceClassification.from_pretrained(
    params['bert_model'],
    cache_dir=params['cache_dir'],
    num_labels=params['num_labels']).to(params['device'])

## Training and evaluation

In [4]:
EPOCH_NUM = 2

params['num_train_epochs'] = 1
checkpoint_files = {
    'config': 'bert_config.json',
    'files_to_save': 'model_{}_epoch_{}.pth'.format(
        params['task_name'], EPOCH_NUM)
}

model, result = train(model, tokenizer, params,
                      train_examples,
                      valid_examples=dev_examples,
                      checkpoint_files=checkpoint_files)
result

***** Running training *****
Num examples: 363849
Batch size:   32
Num steps:    11370


Iteration:   0%|          | 0/11371 [00:00<?, ?it/s]


Epoch: 1


Iteration: 100%|██████████| 11371/11371 [1:24:13<00:00,  2.72it/s]


{'train_loss': 0.2933985063693641}
***** Running evaluation *****
Num examples:  40430
Batch size:    8


Evaluating: 100%|██████████| 5054/5054 [03:13<00:00, 26.19it/s]


{'eval_loss': 0.23927044424873628, 'eval_accuracy': 0.8960425426663369, 'eval_f1_score': 0.8607586549610734, 'eval_matthews_corrcoef': 0.7780305662751378}


{'train_loss': 0.2933985063693641, 'train_global_step': 11371}

In [7]:
EPOCH_NUM = 2

params['num_train_epochs'] = 1
checkpoint_files = {
    'config': 'bert_config.json',
    'files_to_save': 'model_{}_epoch_{}.pth'.format(
        params['task_name'], EPOCH_NUM)
}

model, result = train(model, tokenizer, params,
                      train_examples,
                      valid_examples=dev_examples,
                      checkpoint_files=checkpoint_files)
result

***** Running training *****
Num examples: 363849
Batch size:   32
Num steps:    11370


Iteration:   0%|          | 0/11371 [00:00<?, ?it/s]


Epoch: 1


Iteration: 100%|██████████| 11371/11371 [1:24:15<00:00,  2.72it/s]


{'train_loss': 0.14595894406124307, 'train_global_step': 11371}
***** Running evaluation *****
Num examples:  40430
Batch size:    8


Evaluating: 100%|██████████| 5054/5054 [03:13<00:00, 26.16it/s]


{'eval_loss': 0.23793546136117788, 'eval_accuracy': 0.9007667573583972, 'eval_f1_score': 0.8679828891082593, 'eval_matthews_corrcoef': 0.7889644723506358}


KeyError: 'file_to_save'

In [8]:
result, prob_preds = evaluate(model, tokenizer, params,
                              dev_examples)
result

***** Running evaluation *****
Num examples:  40430
Batch size:    8


Evaluating: 100%|██████████| 5054/5054 [03:12<00:00, 26.25it/s]


{'eval_loss': 0.23793546136117788,
 'eval_accuracy': 0.9007667573583972,
 'eval_f1_score': 0.8679828891082593,
 'eval_matthews_corrcoef': 0.7889644723506358}

In [ ]:
EPOCH_NUM = 3

params['num_train_epochs'] = 1
checkpoint_files = {
    'config': 'bert_config.json',
    'files_to_save': 'model_{}_epoch_{}.pth'.format(
        params['task_name'], EPOCH_NUM)
}

model, result = train(model, tokenizer, params,
                      train_examples,
                      valid_examples=dev_examples,
                      checkpoint_files=checkpoint_files)
result

In [10]:
result

{'eval_loss': 0.23793546136117788,
 'eval_accuracy': 0.9007667573583972,
 'eval_f1_score': 0.8679828891082593,
 'eval_matthews_corrcoef': 0.7889644723506358}

In [11]:
result, prob_preds = evaluate(model, tokenizer, params,
                              dev_examples)
result

***** Running evaluation *****
Num examples:  40430
Batch size:    8


Evaluating: 100%|██████████| 5054/5054 [03:12<00:00, 26.24it/s]


{'eval_loss': 0.2551050326235847,
 'eval_accuracy': 0.8989116992332427,
 'eval_f1_score': 0.8660131790315708,
 'eval_matthews_corrcoef': 0.7855124851154266}